In [1]:
## model modules for total detection model

"""
*module 1: part_symptom_classifying.h5

    * x_data: part image features: bboxes for each plant image -> extract features
    * y_data: part_status(Y: yes symptom, N: no symptom)
    * model pulpose: regardless which is in a certain part of plant, distinguish the status of input par

*module 2: part_disease_class_classifying.h5

    * x_data: same as module 1 but only use 'Y' labeled parts
    * y_data: disease_class data
    * module purpose: regardless which is in a certain part, distinguish the disease class of 'Y' labeled parts

* module 1, 2 integrated structure(after modules trained)(ex. blight)
    *input data: image[bbox1, bbox2,...] -> 이 리스트를 순회하면서 개별 바운딩 박스에 대해 모듈 추론 적용후 결과 병합)
        *module 1 prediction(input: image[])
            *prediction: image[N, Y, N, Y, ...]
            *classifying: 
                if all image[i] == 'N' --> {normal(output directly and start next loop)}
                else                   --> abnormal(go to module 2)

                *module 2 prediction(input: abnormal([N, Y, N, Y])
                    output_list = []
                    *classifying:
                        if abnormal[] == 'N' --> output list.append() 
                        else                 --> output_list.append(predicted_class)

    final_output:
        *(normal): imagename = {bbox: [[bbox1], [bbox2],...], status: [N, N, ...], class: normal}
        *(abnormal): imagename = {bbox: [[bbox1], [bbox2], ...], status: [N, Y, N, Y], class: blight}

"""

"\n*module 1: part_symptom_classifying.h5\n\n    * x_data: part image features: bboxes for each plant image -> extract features\n    * y_data: part_status(Y: yes symptom, N: no symptom)\n    * model pulpose: regardless which is in a certain part of plant, distinguish the status of input par\n\n*module 2: part_disease_class_classifying.h5\n\n    * x_data: same as module 1 but only use 'Y' labeled parts\n    * y_data: disease_class data\n    * module purpose: regardless which is in a certain part, distinguish the disease class of 'Y' labeled parts\n\n* module 1, 2 integrated structure(after modules trained)\n    *input data: image[bbox1, bbox2,...]\n    *module 1 prediction(input: image[])\n        *prediction: image[N, Y, N, Y, ...]\n        *classifying: \n            if all image[i] == 'N' --> {normal(prediction finished)}\n            else                   --> abnormal(go to module 2)\n\n            *module 2 prediction(input: abnormal([N, Y, N, Y])\n                output_list = []

In [3]:
# import lists

import pandas as pd
import numpy
import os
import torch
from torchvision import models, transforms
from PIL import Image
import cv2

In [ ]:
# label data load and define X, Y data

In [ ]:
# part image extraction and image preprocess for CNN model

In [ ]:
# model build

In [ ]:
# model train

In [ ]:
# model validation and save